After creating the database the next step is to create the automatitazion of downloading data from yfinance and finvizfinance and store it in the database

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import yfinance


In [ ]:


def connect_db():


    load_dotenv()  # Cargar variables de entorno desde .env

    USER = os.getenv("DB_USER")
    PASSWORD = os.getenv("DB_PASSWORD")
    HOST = os.getenv("DB_HOST")
    PORT = os.getenv("DB_PORT")
    DB_NAME = os.getenv("DB_NAME")
    

    engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")
    return engine


In [ ]:
def store_historical_prices(ticker):
    """Descarga y almacena precios históricos en PostgreSQL."""
    engine = connect_db()
    
    df = yf.download(ticker, period="5y")
    
    df = df.reset_index()
    df["ticker"] = ticker
    df = df.rename(columns={"Date": "date", "Open": "open_price", "High": "high_price",
                            "Low": "low_price", "Close": "close_price", "Adj Close": "adj_close_price",
                            "Volume": "volume"})
    
    df.to_sql("historical_prices", engine, if_exists="append", index=False)
    print(f"Datos de {ticker} insertados en historical_prices.")


In [ ]:
def store_company_info(ticker):
    """Descarga y almacena información de la empresa en PostgreSQL."""
    engine = connect_db()
    stock = yf.Ticker(ticker)
    info = stock.info
    
    # Crear un DataFrame con la información clave
    df = pd.DataFrame([{
        "ticker": ticker,
        "name": info.get("longName", ""),
        "sector": info.get("sector", ""),
        "industry": info.get("industry", ""),
        "market_cap": info.get("marketCap"),
        "enterprise_value": info.get("enterpriseValue"),
        "total_revenue": info.get("totalRevenue"),
        "ebitda": info.get("ebitda"),
        "net_income": info.get("netIncomeToCommon"),
        "total_debt": info.get("totalDebt"),
        "free_cashflow": info.get("freeCashflow"),
        "return_on_assets": info.get("returnOnAssets"),
        "return_on_equity": info.get("returnOnEquity"),
        "trailing_pe": info.get("trailingPE"),
        "forward_pe": info.get("forwardPE"),
        "price_to_sales": info.get("priceToSalesTrailing12Months"),
        "price_to_book": info.get("priceToBook"),
        "dividend_rate": info.get("dividendRate"),
        "dividend_yield": info.get("dividendYield"),
        "payout_ratio": info.get("payoutRatio"),
        "shares_outstanding": info.get("sharesOutstanding"),
        "held_percent_institutions": info.get("heldPercentInstitutions"),
        "held_percent_insiders": info.get("heldPercentInsiders"),
        "beta": info.get("beta"),
        "fifty_two_week_high": info.get("fiftyTwoWeekHigh"),
        "fifty_two_week_low": info.get("fiftyTwoWeekLow"),
        "earnings_growth": info.get("earningsGrowth"),
        "revenue_growth": info.get("revenueGrowth"),
        "profit_margins": info.get("profitMargins"),
        "gross_margins": info.get("grossMargins"),
        "operating_margins": info.get("operatingMargins"),
        "ebitda_margins": info.get("ebitdaMargins"),
        "target_high_price": info.get("targetHighPrice"),
        "target_low_price": info.get("targetLowPrice"),
        "target_mean_price": info.get("targetMeanPrice"),
        "recommendation_mean": info.get("recommendationMean"),
        "recommendation_key": info.get("recommendationKey"),
        "number_of_analyst_opinions": info.get("numberOfAnalystOpinions"),
    }])


    df.to_sql("company_info", engine, if_exists="append", index=False)
    print(f"Datos de {ticker} insertados en company_info.")
